In [4]:
import webbrowser
import time
import os
import shutil
import copy
import pandas as pd
import re
import csv
import numpy as np
from pandas import DataFrame
import sys
import json
import urllib
from datetime import datetime, timedelta
import requests

from StringIO import StringIO

In [10]:
### Test
def get_token(response_text):
    try:
        return response_text.split('token":"')[1].split('","')[0]
    except:
        return None

def get_csv_request(response_text):
    try:
        return response_text.split('"widgets":')[1].split(',"lineAnno')[0].split('"request":')[1]       
    except:
        return None

#print(requests.get("https://trends.google.com/trends/explore?date=2017-03-26%202017-04-26&q=fashion%20week").text.encode('utf8'))
time_formatted = '2015-01-06' + '+' + '2017-01-09'
req = {"comparisonItem":[{"keyword":'panda', "geo":'US', "time": time_formatted}],"category": 22, "property":""}
hl = "en-GB"
tz = "-120"
#print(json.dumps(req).replace(' ','').replace('+',' '))
explore_URL = 'https://trends.google.com/trends/api/explore?hl={0}&tz={1}&req={2}'.format(hl,tz,json.dumps(req).replace(' ','').replace('+',' '))
#print explore_URL
requests.get(explore_URL).text.encode('utf8')
response_text = requests.get(explore_URL).text
#print(response_text)
request = get_csv_request(response_text)
token = get_token(response_text)
#print(response_text)

csv = requests.get('https://www.google.com/trends/api/widgetdata/multiline/csv?req={0}&token={1}&tz=-120'.format(request,token))
print(csv.text)
print(csv.text.split('\n')[3:4])
text = csv.text.encode('utf8').split('\n')
a = text[3:len(text)-1]
df = pd.DataFrame(columns = ["Date","Volume"])
#for line in a:
#    s = line.split(',')
#    df = df.append([s[0],'panda',s[1]])

def spl (str):
    return str.split(',')

df = pd.DataFrame((map((lambda x: x.split(',')),a)))
df['term'] = 'panda'
df.columns = ['date', 'volume','term']
print(df)


Category: Books & Literature

Week,panda: (United States)
2015-01-11,40
2015-01-18,30
2015-01-25,46
2015-02-01,33
2015-02-08,34
2015-02-15,49
2015-02-22,36
2015-03-01,34
2015-03-08,37
2015-03-15,42
2015-03-22,44
2015-03-29,34
2015-04-05,50
2015-04-12,36
2015-04-19,40
2015-04-26,31
2015-05-03,43
2015-05-10,36
2015-05-17,30
2015-05-24,26
2015-05-31,30
2015-06-07,31
2015-06-14,43
2015-06-21,31
2015-06-28,25
2015-07-05,24
2015-07-12,28
2015-07-19,25
2015-07-26,30
2015-08-02,32
2015-08-09,27
2015-08-16,26
2015-08-23,40
2015-08-30,26
2015-09-06,34
2015-09-13,33
2015-09-20,42
2015-09-27,38
2015-10-04,28
2015-10-11,53
2015-10-18,30
2015-10-25,33
2015-11-01,41
2015-11-08,44
2015-11-15,41
2015-11-22,36
2015-11-29,36
2015-12-06,42
2015-12-13,48
2015-12-20,49
2015-12-27,41
2016-01-03,35
2016-01-10,46
2016-01-17,46
2016-01-24,79
2016-01-31,100
2016-02-07,65
2016-02-14,70
2016-02-21,54
2016-02-28,58
2016-03-06,65
2016-03-13,59
2016-03-20,57
2016-03-27,64
2016-04-03,70
2016-04-10,74
2016-04-17,87
201

In [ ]:
def get_data(bucket_start_date,bucket_end_date, keyword):
    bucket_start_date_printed = datetime.strftime(bucket_start_date, '%Y-%m-%d')
    bucket_end_date_printed = datetime.strftime(bucket_end_date, '%Y-%m-%d')
    time_formatted = bucket_start_date_printed + '+' + bucket_end_date_printed

    req = {"comparisonItem":[{"keyword":keyword, "geo":geo, "time": time_formatted}], "category":category,"property":""}
    hl = "en-GB"
    tz = "-120"

    explore_URL = 'https://trends.google.com/trends/api/explore?hl={0}&tz={1}&req={2}'.format(hl,tz,json.dumps(req).replace(' ','').replace('+',' '))
    print explore_URL
    print requests.get(explore_URL).text.encode('utf8')
    return requests.get(explore_URL).text



In [ ]:
def get_csv(response_text):
    request = get_csv_request(response_text)
    token = get_token(response_text)

    csv = requests.get('https://www.google.com/trends/api/widgetdata/multiline/csv?req={0}&token={1}&tz=-120'.format(request,token))
    return csv.text.encode('utf8')

def parse_csv(csv_contents):
    lines = csv_contents.split('\n')
    df = pd.DataFrame(columns = ['date','value'])
    dates = []
    values = []
    # Delete top 3 lines
    for line in lines[3:]:
        try:
            dates.append(line.split(',')[0].replace(' ',''))
            values.append(line.split(',')[1].replace(' ',''))
        except:
            pass
    df['date'] = dates
    df['value'] = values
    print df.head()
    return df   

In [33]:
### Main

def get_interest_over_time (start_date, end_date, kw_list): 
    geo = 'US'
    category = 22
    final_frame = pd.DataFrame()
    for keyword in kw_list:
        keywords = '+'.join(keyword)
        daily_frames = get_daily_frames(start_date, end_date, keywords)
        weekly_frame = get_weekly_frame(start_date, end_date, keywords)
        temp = stitch_frames(keyword, daily_frames, weekly_frame)
        final_frame = pd.concat([final_frame, temp.reset_index(drop = True)], axis = 1)
        
    return final_frame
kw_list = ['a-line', 'fashion']

out = get_interest_over_time("2015-05-05", "2016-07-05", kw_list)
print out.shape

/Users/nguyentrang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:105: FutureWarning: sort(....) is deprecated, use sort_index(.....)


(147, 2)


In [40]:
start_date = "2015-05-05"
end_date = "2016-07-05"
keyword = "a-line"
geo = 'US'
category = 22
keywords = '+'.join(keyword)
daily_frames = get_daily_frames(start_date, end_date, keywords)
weekly_frame = get_weekly_frame(start_date, end_date, keywords)
final_frame = stitch_frames(keywords, daily_frames, weekly_frame)
print daily_frames[0]

           date volumne
0    2015-05-05       0
1    2015-05-06       0
2    2015-05-07       0
3    2015-05-08       0
4    2015-05-09       0
5    2015-05-10       0
6    2015-05-11       0
7    2015-05-12       0
8    2015-05-13       0
9    2015-05-14       0
10   2015-05-15       0
11   2015-05-16       0
12   2015-05-17       0
13   2015-05-18       0
14   2015-05-19       0
15   2015-05-20       0
16   2015-05-21       0
17   2015-05-22       0
18   2015-05-23       0
19   2015-05-24       0
20   2015-05-25       0
21   2015-05-26       0
22   2015-05-27       0
23   2015-05-28       0
24   2015-05-29       0
25   2015-05-30       0
26   2015-05-31       0
27   2015-06-01       0
28   2015-06-02       0
29   2015-06-03       0
..          ...     ...
152  2015-10-04       0
153  2015-10-05       0
154  2015-10-06       0
155  2015-10-07       0
156  2015-10-08       0
157  2015-10-09       0
158  2015-10-10       0
159  2015-10-11       0
160  2015-10-12       0
161  2015-10-13 

/Users/nguyentrang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:105: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [43]:
print daily_frames[2]

          date volumne
0   2016-05-01       0
1   2016-05-02       0
2   2016-05-03       0
3   2016-05-04       0
4   2016-05-05       0
5   2016-05-06       0
6   2016-05-07       0
7   2016-05-08       0
8   2016-05-09       0
9   2016-05-10       0
10  2016-05-11       0
11  2016-05-12       0
12  2016-05-13       0
13  2016-05-14       0
14  2016-05-15       0
15  2016-05-16       0
16  2016-05-17       0
17  2016-05-18       0
18  2016-05-19       0
19  2016-05-20       0
20  2016-05-21       0
21  2016-05-22       0
22  2016-05-23       0
23  2016-05-24       0
24  2016-05-25       0
25  2016-05-26       0
26  2016-05-27       0
27  2016-05-28       0
28  2016-05-29       0
29  2016-05-30     100
..         ...     ...
36  2016-06-06       0
37  2016-06-07       0
38  2016-06-08       0
39  2016-06-09       0
40  2016-06-10       0
41  2016-06-11       0
42  2016-06-12       0
43  2016-06-13       0
44  2016-06-14       0
45  2016-06-15       0
46  2016-06-16      83
47  2016-06

In [5]:

### Copied program
def get_buckets(start_date, end_date):
    start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    bucket_limits = [start_date_dt]
    left_limit = start_date_dt
    while left_limit <= end_date_dt:
        new_limit = left_limit + timedelta(days=181)
        if new_limit < end_date_dt:
            bucket_limits.append(new_limit)
        left_limit = new_limit
    bucket_limits.append(end_date_dt)
    return bucket_limits

def get_data(bucket_start_date,bucket_end_date, keyword):
    bucket_start_date_printed = datetime.strftime(bucket_start_date, '%Y-%m-%d')
    bucket_end_date_printed = datetime.strftime(bucket_end_date, '%Y-%m-%d')
    time_formatted = bucket_start_date_printed + '+' + bucket_end_date_printed

    req = {"comparisonItem":[{"keyword":keyword, "geo": '', "time": time_formatted}], "category": 22,"property":""}
    hl = "en-GB"
    tz = "-120"

    explore_URL = 'https://trends.google.com/trends/api/explore?hl={0}&tz={1}&req={2}'.format(hl,tz,json.dumps(req).replace(' ','').replace('+',' '))
    return requests.get(explore_URL).text

def get_token(response_text):
    try:
        return response_text.split('token":"')[1].split('","')[0]
    except:
        return None

def get_csv_request(response_text):
    try:
        return response_text.split('"widgets":')[1].split(',"lineAnno')[0].split('"request":')[1]       
    except:
        return None

def get_csv(response_text):
    request = get_csv_request(response_text)
    token = get_token(response_text)

    csv = requests.get('https://www.google.com/trends/api/widgetdata/multiline/csv?req={0}&token={1}&tz=-120'.format(request,token))
    return csv.text.encode('utf8')

def parse_csv(csv_contents):
    lines = csv_contents.split('\n')
    df = pd.DataFrame(columns = ['date','value'])
    dates = []
    values = []
    # Delete top 3 lines
    for line in lines[3:-1]:
        try:
            dates.append(line.split(',')[0].replace(' ',''))
            values.append(line.split(',')[1].replace(' ',''))
        except:
            pass
    df['date'] = dates
    df['value'] = values
    return df   

def get_daily_frames(start_date, end_date, keyword):

    bucket_list = get_buckets(start_date, end_date)
    frames = []
    for i in range(0,len(bucket_list) - 1):
        resp_text = get_data(bucket_list[i], bucket_list[i+1], keyword)
        frames.append(parse_csv(get_csv(resp_text)))

    return frames

def get_weekly_frame(start_date, end_date, keyword):

    if datetime.strptime(start_date, '%Y-%m-%d') > datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=180):
        print 'No need to stitch; your time interval is short enough. '
        return None
    else:
        resp_text = get_data(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d'), keyword)
        return parse_csv(get_csv(resp_text))

def get_weekly_frame2 (start_date, end_date, keyword):
    resp_text = get_data(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d'), keyword)
    return parse_csv(get_csv(resp_text))

def stitch_frames(keywords, daily_frames, weekly_frame):
    if weekly_frame is None:
        return pd.concat(daily_frames, ignore_index = True)
    else: 
        daily_frame = pd.concat(daily_frames, ignore_index = True)
        daily_frame.columns = ['Date', 'Daily_Volume']
        pd.to_datetime(daily_frame['Date']) 
    
        weekly_frame.columns = ['Week_Start_Date', 'Weekly_Volume']
        weekly_frame.index = weekly_frame['Week_Start_Date']
        
        daily_frame.index = daily_frame['Date']
    
        bins = []

        for i in range(0,len(weekly_frame)):
            bins.append(pd.date_range(weekly_frame['Week_Start_Date'][i],periods=7,freq='d'))

        final_data = {}

        for i in range(0,len(bins)):
            week_start_date = datetime.strftime(bins[i][0],'%Y-%m-%d')
            for j in range(0,len(bins[i])):
                this_date = datetime.strftime(bins[i][j],'%Y-%m-%d')
                try:
                    this_val = int(float(weekly_frame['Weekly_Volume'][week_start_date])*float(daily_frame['Daily_Volume'][this_date])/float(daily_frame['Daily_Volume'][week_start_date]))
                    final_data[this_date] = this_val
                except:
                    pass
        final_data_frame = DataFrame.from_dict(final_data,orient='index').sort()
        final_data_frame[0] = np.round(final_data_frame[0]/final_data_frame[0].max()*100,2)

        final_data_frame.columns=['v_'+keywords]
        final_data_frame.index.names = ['Date']

        return final_data_frame




In [2]:
s = "fashion week"
s.replace(' ','+')

'fashion+week'

In [6]:
## Main function
def get_interest_over_time (start_date, end_date, kw_list): 
    geo = ''
    category = 22
    final_frame = pd.DataFrame()
    for keyword in kw_list:
        keywords = keyword.replace(' ','+')
        weekly_frame = get_weekly_frame2(start_date, end_date, keywords)
        final_frame = pd.concat([final_frame, weekly_frame], axis = 1)
    final_frame.index = final_frame.iloc[:,0]
    final_frame = final_frame.drop('date',1)
    final_frame.columns = kw_list
    return final_frame



In [8]:
## Time frame
start_date = '2012-01-01'
end_date = '2016-01-01'
## Event terms
event_term_df = pd.read_csv("event.csv")
event_term = event_term_df['Keyword'].unique().tolist()

## Output csv file with events populuarity index
category_search_df = get_interest_over_time(start_date, end_date, event_term)
print category_search_df
category_search_df = category_search_df.to_csv('event-search.csv')



           paris fashion week academy award milan ss london fashion week  \
date                                                                       
2012-01-01                 48            64        0                   0   
2012-01-08                  0            36        0                   0   
2012-01-15                  0            23        0                  27   
2012-01-22                 45            35        0                  53   
2012-01-29                  0            30        0                  52   
2012-02-05                  0            25       99                  26   
2012-02-12                 89            40        0                  26   
2012-02-19                 44            30        0                  66   
2012-02-26                 66           100        0                  27   
2012-03-04                 87            20        0                   0   
2012-03-11                 86            34        0                  26   
2012-03-18  

In [26]:
## Categories
category_term_df = pd.read_csv("categories.csv")
category_term = category_term_df['Category'].tolist()
category_term = [x.replace(')','') for x in category_term]
category_term = [x.replace('(','') for x in category_term]
linh = category_term[50:100]
jae = category_term [40:50]
linh2 = category_term[10:34]
print linh2
#cateogry_search_df = get_interest_over_time(start_date, end_date, category_term[0])
#print(category_search_df[:100,:7])
#category_search_df = category_search_df.to_csv('category-search.csv')

['Boat neck', 'Boater', 'Bowler hat', 'Buckskins', 'Busby', 'Bustier', 'Bustle', 'Button', 'Camp shirt', 'Chaps', 'Chic style', 'Choker', 'Cleavage breasts', 'Coat clothing', 'Cocktail dress', 'Cope', 'Corsage', 'Corset', 'Cowboy hat', 'Cowl', 'Cuff', 'Cut and sew', 'Dandy', 'Dirndl']


In [54]:
### edge_list
### Calculate the correlation (edge_list)
event_search_df = pd.read_csv("event-search.csv")
category_search_df = pd.read_csv("category-search.csv")
event_term_list = event_search_df.columns[1:]
category_term_list = category_search_df.columns[1:]

event_search_df['date'] = pd.to_datetime(event_search_df['date'])
category_search_df['date'] = pd.to_datetime(category_search_df['date'])
event_search_df['Month'] = event_search_df['date'].apply(lambda x: x.strftime('%B-%Y'))
#category_search_df['Month'] = category_search_df['date'].apply(lambda x: x.strftime('%B-%Y'))


edge_list = pd.DataFrame(columns = ['Month','Category', 'Event', 'Link'])
concat_df = pd.concat([event_search_df, category_search_df], axis = 1)

for category in category_term_list:
    for event in event_term_list:
        temp = concat_df.groupby('Month')[[category, event]].corr().ix[0::2,event]
        temp = temp.reset_index()
        temp.columns = ['Month', 'Category','Link']
        temp['Event'] = event
        edge_list = edge_list.append(temp, ignore_index = True)

edge_list = edge_list.fillna(0)

print edge_list
edge_list.to_csv("edge-list.csv")


ValueError: Grouper for 'Month' not 1-dimensional

In [59]:
### Create node_list
event_search_df = pd.read_csv("event-search.csv")
category_search_df = pd.read_csv("category-search.csv")
event_term_list = event_search_df.columns[1:]
category_term_list = category_search_df.columns[1:]

event_search_df['date'] = pd.to_datetime(event_search_df['date'])
category_search_df['date'] = pd.to_datetime(category_search_df['date'])
event_search_df['Month'] = event_search_df['date'].apply(lambda x: x.strftime('%B-%Y'))
category_search_df['Month'] = category_search_df['date'].apply(lambda x: x.strftime('%B-%Y'))




## For the event
event_search_df = event_search_df.drop('date',1)
event_search_df.index = event_search_df['Month']
event_search_df = event_search_df.drop('Month',1)
event_search_df = event_search_df.unstack().reset_index()
event_search_df.columns = ['event', 'Month', 'value']
event_search_df = event_search_df.groupby(['Month', 'event'], as_index = False).mean()
event_search_df.columns = ['Month','Node', 'Popularity']
event_search_df['Type'] = 'Event'




In [60]:
## For the category
category_search_df = pd.read_csv("category-search.csv")
category_search_df['date'] = pd.to_datetime(category_search_df['date'])
category_search_df['Month'] = category_search_df['date'].apply(lambda x: x.strftime('%B-%Y'))


category_search_df = category_search_df.drop('date',1)
category_search_df.index = category_search_df['Month']
category_search_df = category_search_df.drop('Month',1)
category_search_df = category_search_df.unstack().reset_index()
category_search_df.columns = ['event', 'Month', 'value']
category_search_df = category_search_df.groupby(['Month', 'event'], as_index = False).mean()
category_search_df.columns = ['Month','Node', 'Popularity']
category_search_df['Type'] = 'Clothing'



### Merge 2 node lists into one
node_list = category_search_df.append(event_search_df)
node_list.to_csv("node_list.csv")
print node_list

              Month                              Node  Popularity      Type
0        April-2012                         Boat neck       38.20  Clothing
1        April-2012                            Boater       39.00  Clothing
2        April-2012                        Bowler hat       58.40  Clothing
3        April-2012                         Buckskins       33.80  Clothing
4        April-2012                             Busby       44.20  Clothing
5        April-2012                           Bustier       91.20  Clothing
6        April-2012                            Button       87.40  Clothing
7        April-2012                        Camp shirt       67.40  Clothing
8        April-2012                             Chaps       78.00  Clothing
9        April-2012                        Chic style       73.00  Clothing
10       April-2012                            Choker       21.80  Clothing
11       April-2012                   Cleavage breast       38.60  Clothing
12       Apr